In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Paamiut,GL,2020-08-01 15:53:31,61.99,-49.67,45.99,91,100,12.37
1,1,Mahebourg,MU,2020-08-01 15:53:31,-20.41,57.70,73.40,78,75,10.29
2,2,Busselton,AU,2020-08-01 15:53:31,-33.65,115.33,48.00,78,78,3.09
3,3,Beloha,MG,2020-08-01 15:53:31,-25.17,45.05,61.59,70,0,10.63
4,4,Chimore,BO,2020-08-01 15:53:31,-16.98,-65.13,89.37,35,75,0.94


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds , dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind , dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 55


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Paamiut,GL,2020-08-01 15:53:31,61.99,-49.67,45.99,91,100,12.37
2,2,Busselton,AU,2020-08-01 15:53:31,-33.65,115.33,48.00,78,78,3.09
7,7,Tasiilaq,GL,2020-08-01 15:53:31,65.61,-37.64,53.60,55,20,5.82
8,8,Ancud,CL,2020-08-01 15:53:32,-41.87,-73.82,52.00,87,90,17.22
11,11,Mataura,NZ,2020-08-01 15:53:32,-46.19,168.86,47.68,86,73,4.59
...,...,...,...,...,...,...,...,...,...,...
498,498,Havoysund,NO,2020-08-01 15:54:41,71.00,24.66,50.00,81,75,6.93
499,499,Uyuni,BO,2020-08-01 15:54:42,-20.46,-66.83,53.98,31,12,5.44
502,502,Geraldton,AU,2020-08-01 15:54:42,-28.77,114.60,50.00,87,94,10.29
553,553,Port Augusta,AU,2020-08-01 15:54:49,-32.50,137.77,50.00,53,100,8.05


In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Paamiut,GL,45.99,61.99,-49.67,
2,Busselton,AU,48.00,-33.65,115.33,
7,Tasiilaq,GL,53.60,65.61,-37.64,
8,Ancud,CL,52.00,-41.87,-73.82,
11,Mataura,NZ,47.68,-46.19,168.86,
12,Bluff,NZ,52.23,-46.60,168.33,
25,Tuktoyaktuk,CA,53.60,69.45,-133.04,
34,Ulladulla,AU,50.00,-35.35,150.47,
49,Mar Del Plata,AR,53.60,-38.00,-57.56,
53,Kodiak,US,48.20,57.79,-152.41,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [13]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
hotels

{'html_attributions': [],
 'next_page_token': 'CqQCGwEAAKVqpVPu_pd0uJGvxtFNI-oL6xxeqBv0X9IPIWAGBupvP9wwLRSkxxPqZ6xuvatHQsaS_Q_hu6Afm3GW4QP1nGnspoMphB3sVA7GYI_jsbd5qD1k8zdkosPltMDHmcBjG8sAZxGTrUM6JQYp_IcZCoOIom0ZdMJ7Kau8Ye2FDZViFPyZJSGIHcczyEhqbIJOvwkRAMEnvKik4wAYA3S0RnmEcsfPfUfIs1_32uP4XNutY80iG_cMxCXhyc54z1-3AnRYXIf5DosmaO9F_9FEzAa-xRTsr5hzNECeZPbdmqjXKWp03Psw2vOIwbsdC8lXQ14N8E3SouNohA6kq-_DVZz17E5AB2ESWoxt94h1zqv19J9rPpP6lcvEjmQXB25JzRIQ5KG2-biLXNyqt9Ri5geYXBoUf2MTbd2lSUrqCiTuznH5Wl1-Quw',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291502},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))